In [1]:
import numpy
from matplotlib import pyplot
%matplotlib inline

In [2]:
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

In [3]:
def rho_shock_tube(x, rhoL, rhoR):
    rho = numpy.zeros_like(x)
    mask1 = numpy.where(x < 0.0)
    mask2 = numpy.where(0.0 <= x)
    rho[mask1] = rhoL
    rho[mask2] = rhoR
    return rho

In [4]:
def u_shock_tube(x, uL, uR):
    u = numpy.zeros_like(x)
    mask1 = numpy.where(x < 0.0)
    mask2 = numpy.where(0.0 <= x)
    u[mask1] = uL
    u[mask2] = uR
    return u

In [5]:
def P_shock_tube(x, PL, PR):
    P = numpy.zeros_like(x)
    mask1 = numpy.where(x < 0.0)
    mask2 = numpy.where(0.0 <= x)
    P[mask1] = PL
    P[mask2] = PR
    return P

In [6]:
nx = 81
L = 10.0
dx = 0.25
dt = 0.0002
gamma = 1.4
rhoL = 1.0
rhoR = 0.125
uL = 0.0
uR = 0.0
PL = 100000.0
PR = 10000.0
nt = 50

x = numpy.linspace(-L, L, num=nx)

rho0 = rho_shock_tube(x, rhoL, rhoR)
u0 = u_shock_tube(x, uL, uR)
P0 = P_shock_tube(x, PL, PR)

In [7]:
def e_def(P, rho, gamma):
    e = P / ((gamma - 1) * rho)
    return e

def U_def(rho, u, P, e):
    U = numpy.array([rho.copy(),
                     u.copy() * rho.copy(),
                     P.copy() * (e * 0.5 * u.copy()**2)])
    return U

def U_def(rho, u):
    U = numpy.array([u.copy() * rho.copy()])
    return U

In [8]:
e0 = e_def(P0, rho0, gamma)

U0 = U_def(rho0, u0, P0, e0)

U0 = U_def(rho0, u0)

U0

def flux(U, gamma, rho, u, P, e, *args):
    F = numpy.zeros_like(U)
    F[0] = U[1]
    F[1] = U[1]**2 / U[0] + (gamma - 1) * (U[2] - 0.5 * U[1]**2 / U[0])
    F[2] = (U[2] + (gamma - 1) * (U[2] - 0.5 * U[1]**2 / U[0])) * U[1] / U[0]
    return F

In [9]:
def flux(rho, u):
    F = rho * u
    return F

def rich(U0, nt, gamma, dt, dx, bc_values, rho, u, P, e, *args):
    U_hist = [U0.copy()]
    U = U0.copy()
    U_half = U.copy()
    for n in range(nt):
        F = flux(U, gamma, rho, u, P, e, *args)
        U_half[:,1:-1] = 0.5 * (U[:,2:] + U[:,:-2]) - dt / 2*dx * (F[:,2:] - F[:,:-2])
        F = flux(U_half, gamma, rho, u, P, e, *args)
        U[:,1:-1] = U[:,1:-1] - dt / dx * (F[:,2:] - F[:,:-2])
        U[:,0] = bc_values[0]
        U[:,-1] = bc_values[1]
        U_hist.append(U.copy())
    return U_hist

In [10]:
def minmod(e, dx):
    sigma = numpy.zeros_like(e)
    for i in range(1, len(e) - 1):
        de_minus = (e[i] - e[i - 1]) / dx
        de_plus = (e[i + 1] - e[i]) / dx
        if de_minus > 0 and de_plus > 0:
            sigma[i] = min(de_minus, de_plus)
        elif de_minus < 0 and de_plus < 0:
            sigma[i] = max(de_minus, de_plus)
        else:
            sigma[i] = 0.0
    return sigma

In [11]:
def muscl(rho0, u0, nt, gamma, dt, dx, bc_values):
    def compute_flux(rho, u):
        sigma = minmod(rho, dx)
        rhoL = (rho + sigma * dx / 2.0)
        rhoR = (rho - sigma * dx / 2.0)
        F = 0.5 * (flux(rhoL, u) + flux(rhoR, u) -
                   dx / dt * (rhoR - rhoL))
        return F
    rho = rho0.copy()
    u = u0.copy()
    rho_half = rho.copy()
    for n in range(nt):
        F = compute_flux(rho, u)
        rho_half[1:-1] = 0.5 * (rho[2:] + rho[:-2]) - dt / (2*dx) * (F[2:] - F[:-2])
        rho_half[0], rho_half[-1] = bc_values
        F = compute_flux(rho_half, u)
        rho[1:-1] = rho[1:-1] - dt / dx * (F[2:] - F[:-2])
        rho[0], rho[-1] = bc_values
    return rho

In [12]:
rho = muscl(rho0, u0, nt, gamma, dt, dx, (rho0[0], rho0[-1]))

In [13]:
rho

array([1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   ,
       1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   ,
       1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   ,
       1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   ,
       1.   , 1.   , 1.   , 1.   , 0.125, 0.125, 0.125, 0.125, 0.125,
       0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125,
       0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125,
       0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125,
       0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125])

In [20]:
def change_U(u, bc_values):
    for n in range(nt):
        rho = rho0.copy()
        u = u0.copy()
        u_half = u.copy()
        F = flux(rho, u)
        u_half[1:-1] = 0.5 * (u[2:] + u[:-2]) - dt / 2*dx * (F[2:] - F[:-2])
        F = flux(u_half, u)
        u[1:-1] = u[1:-1] - dt / dx * (F[2:] - F[:-2])
        u[0] = bc_values[0]
        u[-1] = bc_values[1]
    return u

In [22]:
u = change_U(u0, (u0[0], u0[-1]))
u

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])